In [8]:


from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


# Function to ask questions
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



/home/meir/Documents/dev/simple-rag/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the variables using os.getenv
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [10]:

db = Chroma(persist_directory="./db-garanties2",
            embedding_function=embeddings)

# # Create retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 5}
)

In [11]:
# local_llm = 'gemma2'
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

# llm = ChatOllama(model=local_llm,
#                  keep_alive="3h", 
#                  max_tokens=512,  
#                  temperature=0)

# Create prompt template
template = """<bos><start_of_turn>user\nAnswer the question based only on the following context and extract out a meaningful answer. \
Please write in full sentences with correct spelling and punctuation. if it makes sense use lists. \
If the context doen't contain the answer, just respond that you are unable to find an answer. \
Use the question language to respond. \

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model\n
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)


In [13]:
def ask_question(question):
    print("Answer:\n\n", end=" ", flush=True)
    for chunk in rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n")

# Example usage
if __name__ == "__main__":
    while True:
        user_question = input("Ask a question (or type 'quit' to exit): ")
        if user_question.lower() == 'quit':
            break
        answer = ask_question(user_question)
        # print("\nFull answer received.\n")

Answer:

 Based on the provided context, the following types of guarantees are proposed:

* For audiologists:
	+ AIDES AUDITIVES: includes guarantees for equipment and accessories, with a limit of 1700 € per equipment every 4 years
	+ Accessoires, entretien et piles are covered by the Sécurité sociale at 100% TM
* For opticians:
	+ OPTIQUE: includes guarantees for equipment and accessories, with a limit of 100% PLV (Prix Limite de Vente) and under deduction of the amount reimbursed by the Sécurité sociale
	+ "100% santé" guarantee: includes coverage of 100% of the costs of equipment and accessories, with a limit of the PLV
	+ Panier maitrisé: a package of essential equipment with a moderate out-of-pocket expense
* For dentists:
	+ Soins et prothèses 100% Santé: includes coverage of 100% of the costs of dental prosthesis, with a limit of the HLF (Honoraire Limite de Facturation)
	+ Soins et prothèses autres que les "Soins et prothèses 100% Santé": includes coverage of dental prosthesis 